In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT * FROM album
LIMIT 1

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1


In [4]:
%%sql
SELECT * FROM artist
LIMIT 1

Done.


artist_id,name
1,AC/DC


In [5]:
%%sql
SELECT * FROM customer
LIMIT 1

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3


In [7]:
%%sql
SELECT * FROM employee
LIMIT 1

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com


In [8]:
%%sql
SELECT * FROM genre
LIMIT 1

Done.


genre_id,name
1,Rock


In [9]:
%%sql
SELECT * FROM invoice
LIMIT 1

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84


In [10]:
%%sql
SELECT * FROM invoice_line
LIMIT 1

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1


In [11]:
%%sql
SELECT * FROM media_type
LIMIT 1

Done.


media_type_id,name
1,MPEG audio file


In [12]:
%%sql
SELECT * FROM playlist
LIMIT 1

Done.


playlist_id,name
1,Music


In [13]:
%%sql
SELECT * FROM playlist_track
LIMIT 1

Done.


playlist_id,track_id
1,3402


In [14]:
%%sql
SELECT * FROM track
LIMIT 1

Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99


Number of tracks sold in USA for each genre in absolute numbers and percentages:

In [9]:
%%sql
WITH tracks_country_usa AS
    (SELECT
        SUM(il.quantity) tracks_sold,
         i.billing_country country
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     GROUP BY 2
     HAVING i.billing_country = "USA"
    )

SELECT g.name genre_name,
    SUM(il.quantity) tracks_sold,
    CAST(SUM(il.quantity) AS float) / CAST(tc.tracks_sold AS float) * 100 tracks_pct
FROM invoice_line il
LEFT JOIN track t ON t.track_id = il.track_id
LEFT JOIN genre g ON g.genre_id = t.genre_id
INNER JOIN invoice i ON i.invoice_id = il.invoice_id
INNER JOIN tracks_country_usa tc ON tc.country = i.billing_country
WHERE i.billing_country = "USA"
GROUP BY 1
ORDER BY 2 DESC;

Done.


genre_name,tracks_sold,tracks_pct
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


Rock, punk, and alternative music make up the majority of music sold by Chinook. Thus the album Red Tone, which is punk, is most likely to sell well in the USA.

In [11]:
%%sql

WITH total_per_customer AS
    (SELECT 
        SUM(i.total) total,
        c.customer_id customer_id,
        c.support_rep_id employee_id
     FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.support_rep_id
    GROUP BY 2
    )

SELECT
    SUM(t.total) total,
    e.employee_id employee_id,
    e.title title,
    e.country country
FROM employee e
INNER JOIN total_per_customer t ON t.employee_id = e.employee_id
GROUP BY 2
ORDER BY 1;

Done.


total,employee_id,title,country
1445.3999999999999,4,Sales Support Agent,Canada
2099.79,3,Sales Support Agent,Canada
2601.72,5,Sales Support Agent,Canada


In [19]:
%%sql

WITH country_other AS
    (SELECT
        CASE WHEN(
                SELECT COUNT(*)
                FROM customer
                WHERE country = c.country
                ) = 1 THEN "other"
            ELSE c.country
        END AS country,
        c.customer_id,
        il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT country, customers, total_sales, average_order_value, customer_lifetime_value
FROM
    (SELECT
        country,
        COUNT(DISTINCT customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / COUNT(DISTINCT customer_id) customer_lifetime_value,
        SUM(unit_price) / COUNT(DISTINCT invoice_id) average_order_value,
        CASE
            WHEN country = "other" THEN 1
            ELSE 0
        END AS sort
    FROM country_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order_value,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
other,15,1094.9400000000085,7.448571428571486,72.99600000000056


In [22]:
%%sql

WITH invoice_first_track AS
    (
    SELECT
        il.invoice_id invoice_id,
        MIN(il.track_id) first_track_id
    FROM invoice_line il
    GROUP BY 1
    )
    
SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CASE(COUNT(invoice_id) AS FLOAT) / (
                                        SELECT COUNT(*) FROM invoice
                                

(sqlite3.OperationalError) near "AS": syntax error
[SQL: WITH invoice_first_track AS
    (
    SELECT
        il.invoice_id invoice_id,
        MIN(il.track_id) first_track_id
    FROM invoice_line il
    GROUP BY 1
    )
    
SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CASE(COUNT(invoice_id) AS FLOAT) / (
                                        SELECT COUNT(*) FROM invoice
                                    ) percent
    FROM 
        (
        SELECT
            ifs.*,
            CASE
                WHEN
                    (
                    SELCT t.track_id FROM track t
                    WHERE t.album_id = (
                                        SELECT t2.album_id FROM track t2
                                        WHERE t2.track_id = ifs.first_track_id
                                        )
                    
                    EXCEPT
                    
                    SELECT il2.track_id FROM invoice_line il2
                  